### Lets create a few group by views that could help us, ie sign up cohorts, provider, country 
- Lets start by loading in the customer data we enriched with metrics

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

In [8]:
# Path to your SQLite database
db_path = "Subscriptions.db"

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Find all metrics subscription data tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name LIKE 'metrics_subscription_data_%';")
tables = cursor.fetchall()

if not tables:
    print("No metrics_subscription_data tables found in the database.")
else:
    # Extract table names and sort them to find the most recent one
    metrics_tables = [table[0] for table in tables]
    latest_metrics_table = sorted(metrics_tables)[-1]
    
    print(f"Loading data from the most recent metrics table: '{latest_metrics_table}'")
    
    # Query to get all data from the most recent table
    query = f"SELECT * FROM {latest_metrics_table}"
    
    # Load the data into a pandas DataFrame
    df = pd.read_sql_query(query, conn)
    
    # Display basic information about the DataFrame
    print(f"\nLoaded {len(df)} rows and {len(df.columns)} columns")
    print("\nDataFrame shape:", df.shape)
    print("\nColumn names:", list(df.columns))
    print("\nFirst 5 rows:")
    print(df.head())
    
    # Show summary statistics for key metrics columns
    print("\nKey metrics summary:")
    metric_columns = ['payment_periods', 'avg_monthly_payment', 'expansion_ratio', 'has_expanded', 'current_mrr']
    print(df[metric_columns].describe())

# Close the connection
conn.close()

Loading data from the most recent metrics table: 'metrics_subscription_data_20230116'

Loaded 134610 rows and 21 columns

DataFrame shape: (134610, 21)

Column names: ['oid', 'provider', 'total_charges', 'signup_date', 'is_canceled', 'is_active', 'is_delinquent', 'conversion_date', 'cancellation_date', 'current_mrr', 'personal_person_geo_country', 'converted', 'payment_periods', 'avg_monthly_payment', 'expansion_ratio', 'has_expanded', 'price_tier', 'conversion_speed', 'conversion_speed_category', 'conversion_cohort', 'signup_cohort']

First 5 rows:
               oid provider  total_charges          signup_date  is_canceled  \
0  273624174263463    apple          10.94  2021-04-20 00:00:00            1   
1  532534633483904    apple          10.92  2021-04-20 00:00:00            1   
2  623527156674739    apple           0.00  2021-04-20 00:00:00            0   
3  441616357320658    apple          90.30  2021-04-20 00:00:00            0   
4  189644194376891    apple          79.98  

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134610 entries, 0 to 134609
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   oid                          134610 non-null  object 
 1   provider                     134610 non-null  object 
 2   total_charges                134610 non-null  float64
 3   signup_date                  134610 non-null  object 
 4   is_canceled                  134610 non-null  int64  
 5   is_active                    134610 non-null  int64  
 6   is_delinquent                134610 non-null  int64  
 7   conversion_date              80759 non-null   object 
 8   cancellation_date            33518 non-null   object 
 9   current_mrr                  134610 non-null  float64
 10  personal_person_geo_country  105503 non-null  object 
 11  converted                    134610 non-null  int64  
 12  payment_periods              80759 non-null   float64
 13 

## Lets start by creating a group by for all our possible views. This can be filtered and reduced for answering questions later 
- going to do a seperate table for the sign-up cohort and the conversion cohort

## Group by Table

### We want to include the following group by columns:
- Provider 
- is active
- is canceled
- converted
- personal_person_geo_country
- price_tier
- conversion_speed_category
- conversion cohort
- signup cohort
***Customer count

### We want to include the following aggregations:
- sum(total_charges)
- avg(total_charges)
- avg(current_mrr)
- avg(payment periods)
- avg(avg_monhtly_payment)
- sum(avg_monthly_payments)
- avg(expansion_ratio)
- avg(conversion_speed)

In [ ]:
# Create group by view with multiple dimensions and aggregations


# First, handle missing values in grouping columns to ensure proper grouping
fill_values = {
    'personal_person_geo_country': 'Unknown',
    'price_tier': 'No Tier',
    'conversion_speed_category': 'Not Converted',
    'conversion_cohort': 'Not Converted'
}

# Create a copy of the dataframe for grouping
df_for_grouping = df.copy()

# Replace NaNs with specified values for grouping columns
for col, fill_value in fill_values.items():
    df_for_grouping[col] = df_for_grouping[col].fillna(fill_value)

# Define grouping columns
grouping_columns = [
    'provider', 
    'is_active', 
    'is_canceled', 
    'converted', 
    'personal_person_geo_country', 
    'price_tier', 
    'conversion_speed_category', 
    'conversion_cohort', 
    'signup_cohort'
]

# Group by multiple columns and calculate aggregations
grouped_df = df_for_grouping.groupby(grouping_columns).agg({
    'oid': 'count',                    # count of customers in each group
    'total_charges': ['sum', 'mean'],  # sum and avg
    'current_mrr': 'mean',             # avg
    'payment_periods': 'mean',         # avg
    'avg_monthly_payment': ['mean', 'sum'],  # avg and sum
    'expansion_ratio': 'mean',         # avg
    'conversion_speed': 'mean'         # avg
})

# First flatten the column names
grouped_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in grouped_df.columns.values]

# Now reset the index to turn grouping columns into regular columns
grouped_df = grouped_df.reset_index()

# Rename columns for clarity
grouped_df = grouped_df.rename(columns={
    'oid_count': 'customer_count',
    'total_charges_sum': 'total_charges_sum',
    'total_charges_mean': 'avg_total_charges',
    'current_mrr_mean': 'avg_current_mrr',
    'payment_periods_mean': 'avg_payment_periods',
    'avg_monthly_payment_mean': 'avg_of_avg_monthly_payment',
    'avg_monthly_payment_sum': 'sum_avg_monthly_payment',
    'expansion_ratio_mean': 'avg_expansion_ratio',
    'conversion_speed_mean': 'avg_conversion_speed'
})

# Sort by total charges (descending) to see highest-value segments first
grouped_df = grouped_df.sort_values('total_charges_sum', ascending=False)

# Check the column names (for debugging)
print("Column names in grouped_df:", grouped_df.columns.tolist())

# Display size of result set
print(f"Generated {len(grouped_df)} groups with {grouped_df['customer_count'].sum()} total customers")

# Display the first 10 rows (highest total charges)
print("\nTop 10 groups by total charges:")
pd.set_option('display.max_columns', None)  # Show all columns
display(grouped_df.head(10))



Column names in grouped_df: ['provider', 'is_active', 'is_canceled', 'converted', 'personal_person_geo_country', 'price_tier', 'conversion_speed_category', 'conversion_cohort', 'signup_cohort', 'customer_count', 'total_charges_sum', 'avg_total_charges', 'avg_current_mrr', 'avg_payment_periods', 'avg_of_avg_monthly_payment', 'sum_avg_monthly_payment', 'avg_expansion_ratio', 'avg_conversion_speed']
Generated 16025 groups with 134610 total customers

Top 10 groups by total charges:


,provider,is_active,is_canceled,converted,personal_person_geo_country,price_tier,conversion_speed_category,conversion_cohort,signup_cohort,customer_count,total_charges_sum,avg_total_charges,avg_current_mrr,avg_payment_periods,avg_of_avg_monthly_payment,sum_avg_monthly_payment,avg_expansion_ratio,avg_conversion_speed
12413,apple,1,0,1,United States of America,$20+,1-7 days,2022-11,2022-11,556,33362.43,60.004371,5.000000,2.001799,29.981808,16669.885000,0.166791,6.996403
12300,apple,1,0,1,United States of America,$17.50-$20,1-7 days,2022-10,2022-10,553,33174.47,59.990000,5.000000,3.000000,19.996667,11058.156667,0.250042,7.000000
12579,apple,1,0,1,United States of America,$7.50-$10,1-7 days,2022-07,2022-07,454,27235.46,59.990000,5.000000,6.297357,9.573609,4346.418333,0.524867,6.993392
12415,apple,1,0,1,United States of America,$20+,1-7 days,2022-12,2022-12,412,24715.88,59.990000,5.000000,1.733010,38.003374,15657.390000,0.144442,7.000000
12567,apple,1,0,1,United States of America,$7.50-$10,1-7 days,2021-11,2021-11,202,24236.92,119.984752,5.014802,14.381188,8.352538,1687.212619,0.601132,7.000000
12462,apple,1,0,1,United States of America,$5-$7.50,1-7 days,2022-03,2022-03,380,22798.25,59.995395,5.007868,10.339474,5.814403,2209.473091,0.862962,6.997368
12569,apple,1,0,1,United States of America,$7.50-$10,1-7 days,2021-12,2021-12,190,22780.09,119.895211,5.015737,13.436842,8.934729,1697.598462,0.562374,6.973684
12271,apple,1,0,1,United States of America,$12.50-$15,1-7 days,2022-09,2022-09,378,22676.22,59.990000,5.000000,4.000000,14.997500,5669.055000,0.333389,7.000000
12466,apple,1,0,1,United States of America,$5-$7.50,1-7 days,2022-05,2022-05,367,22016.33,59.990000,5.000000,8.376022,7.185451,2637.060417,0.698118,7.000000
12464,apple,1,0,1,United States of America,$5-$7.50,1-7 days,2022-04,2022-04,334,20036.66,59.990000,5.000000,9.455090,6.362213,2124.979111,0.788055,6.979042


In [13]:
# Example: Filter to only active customers for a specific provider
active_apple = grouped_df[(grouped_df['provider'] == 'apple') & (grouped_df['is_active'] == 1)]
print(f"\nActive Apple customers: {active_apple['customer_count'].sum()} in {len(active_apple)} groups")
display(active_apple.head(5))


Active Apple customers: 39906 in 6333 groups


,provider,is_active,is_canceled,converted,personal_person_geo_country,price_tier,conversion_speed_category,conversion_cohort,signup_cohort,customer_count,total_charges_sum,avg_total_charges,avg_current_mrr,avg_payment_periods,avg_of_avg_monthly_payment,sum_avg_monthly_payment,avg_expansion_ratio,avg_conversion_speed
12413,apple,1,0,1,United States of America,$20+,1-7 days,2022-11,2022-11,556,33362.43,60.004371,5.000000,2.001799,29.981808,16669.885000,0.166791,6.996403
12300,apple,1,0,1,United States of America,$17.50-$20,1-7 days,2022-10,2022-10,553,33174.47,59.990000,5.000000,3.000000,19.996667,11058.156667,0.250042,7.000000
12579,apple,1,0,1,United States of America,$7.50-$10,1-7 days,2022-07,2022-07,454,27235.46,59.990000,5.000000,6.297357,9.573609,4346.418333,0.524867,6.993392
12415,apple,1,0,1,United States of America,$20+,1-7 days,2022-12,2022-12,412,24715.88,59.990000,5.000000,1.733010,38.003374,15657.390000,0.144442,7.000000
12567,apple,1,0,1,United States of America,$7.50-$10,1-7 days,2021-11,2021-11,202,24236.92,119.984752,5.014802,14.381188,8.352538,1687.212619,0.601132,7.000000


In [14]:
# Example: Filter to only active customers for a specific provider
canceled_apple = grouped_df[(grouped_df['provider'] == 'apple') & (grouped_df['is_canceled'] == 1)]
print(f"\nActive Apple customers: {canceled_apple['customer_count'].sum()} in {len(canceled_apple)} groups")
display(canceled_apple.head(5))


Active Apple customers: 29143 in 4883 groups


,provider,is_active,is_canceled,converted,personal_person_geo_country,price_tier,conversion_speed_category,conversion_cohort,signup_cohort,customer_count,total_charges_sum,avg_total_charges,avg_current_mrr,avg_payment_periods,avg_of_avg_monthly_payment,sum_avg_monthly_payment,avg_expansion_ratio,avg_conversion_speed
6037,apple,0,1,1,United States of America,$2.50-$5,1-7 days,2021-12,2021-12,240,14301.59,59.589958,0.0,12.925000,4.603903,1104.936795,0.0,6.983333
6035,apple,0,1,1,United States of America,$2.50-$5,1-7 days,2021-11,2021-11,214,12787.34,59.753925,0.0,12.780374,4.679869,1001.491923,0.0,7.000000
6406,apple,0,1,1,United States of America,$7.50-$10,Not Converted,2022-01,2022-01,443,10147.30,22.905869,0.0,2.866817,7.990000,3539.570000,0.0,0.000000
6404,apple,0,1,1,United States of America,$7.50-$10,Not Converted,2021-11,2021-11,397,9827.70,24.754912,0.0,3.098237,7.990000,3172.030000,0.0,0.000000
6033,apple,0,1,1,United States of America,$2.50-$5,1-7 days,2021-10,2021-10,160,9602.33,60.014562,0.0,12.031250,4.989971,798.395333,0.0,7.000000


In [16]:
# Example: Filter to only active customers for a specific provider
non_converted = grouped_df[(grouped_df['converted'] == 0)]
print(f"\nActive Apple customers: {non_converted['customer_count'].sum()} in {len(non_converted)} groups")
display(non_converted.head(5))


Active Apple customers: 53851 in 2089 groups


,provider,is_active,is_canceled,converted,personal_person_geo_country,price_tier,conversion_speed_category,conversion_cohort,signup_cohort,customer_count,total_charges_sum,avg_total_charges,avg_current_mrr,avg_payment_periods,avg_of_avg_monthly_payment,sum_avg_monthly_payment,avg_expansion_ratio,avg_conversion_speed
14715,stripe,0,0,0,Unknown,No Tier,Not Converted,Not Converted,2022-11,1510,1291.44,0.855258,0.0,NaN,NaN,0.0,NaN,NaN
14703,stripe,0,0,0,Unknown,No Tier,Not Converted,Not Converted,2021-11,2373,1220.49,0.514324,0.0,NaN,NaN,0.0,NaN,NaN
14704,stripe,0,0,0,Unknown,No Tier,Not Converted,Not Converted,2021-12,3627,876.57,0.241679,0.0,NaN,NaN,0.0,NaN,NaN
14714,stripe,0,0,0,Unknown,No Tier,Not Converted,Not Converted,2022-10,991,786.63,0.793774,0.0,NaN,NaN,0.0,NaN,NaN
12956,google,0,0,0,United States of America,No Tier,Not Converted,Not Converted,2022-11,19,708.51,37.290000,0.0,NaN,NaN,0.0,NaN,NaN


### Now lets load the new grouped df into our db so we can do our final analysis with the customer level and grouped df tables

In [24]:


# Path to your SQLite database
db_path = "Subscriptions.db"  # Using the same path from your notebook

# Connect to the SQLite database (won't overwrite if it exists)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Finding the latest signup date in the dataframe and formatting it for the table name
latest_signup_date = grouped_df['signup_cohort'].max()
formatted_date = latest_signup_date.replace("-", "_")  # Replace dashes with underscores for the table name

# Define the new table name with the metrics prefix
table_name = f"grouped_metrics_subscription_data_{formatted_date}"

# Check if this table already exists
cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';")
table_exists = cursor.fetchone()

if table_exists:
    print(f"Table '{table_name}' already exists in the database. Overwriting")
    #print("If you want to replace it, you can use if_exists='replace' parameter.")
else:
    # Save the dataframe to the database
    grouped_df.to_sql(name=table_name, con=conn, if_exists='replace', index=False)
    print(f"Data successfully saved to table '{table_name}' in {db_path}")

# Get information about the saved table (number of rows)
cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
row_count = cursor.fetchone()[0]
print(f"Total rows in '{table_name}': {row_count}")

# Get column information
cursor.execute(f"PRAGMA table_info({table_name});")
columns = cursor.fetchall()
print(f"\nTotal columns: {len(columns)}")
print("\nColumn information:")
for col in columns:
    print(f"  {col[1]} ({col[2]})")

# Close the connection
conn.close()

Table 'grouped_metrics_subscription_data_2023_01' already exists in the database. Overwriting
Total rows in 'grouped_metrics_subscription_data_2023_01': 16025

Total columns: 18

Column information:
  provider (TEXT)
  is_active (INTEGER)
  is_canceled (INTEGER)
  converted (INTEGER)
  personal_person_geo_country (TEXT)
  price_tier (TEXT)
  conversion_speed_category (TEXT)
  conversion_cohort (TEXT)
  signup_cohort (TEXT)
  customer_count (INTEGER)
  total_charges_sum (REAL)
  avg_total_charges (REAL)
  avg_current_mrr (REAL)
  avg_payment_periods (REAL)
  avg_of_avg_monthly_payment (REAL)
  sum_avg_monthly_payment (REAL)
  avg_expansion_ratio (REAL)
  avg_conversion_speed (REAL)


In [25]:

# Path to your SQLite database
db_path = "Subscriptions.db"  # Adjust to your actual database path

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 1. List all tables in the database
print("All tables in the database:")
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(table[0])

# 2. Get the head of your newly created table
# (assuming the table name pattern is subscription_data_YYYYMMDD)
# Find the subscription_data table
subscription_tables = [table[0] for table in tables if table[0].startswith('grouped_metrics_subscription_data_')]

if subscription_tables:
    # Get the most recent table (should be the one you just created)
    latest_table = sorted(subscription_tables)[-1]
    
    print(f"\nFirst 5 rows from table '{latest_table}':")
    cursor.execute(f"SELECT * FROM {latest_table} LIMIT 5;")
    
    # Get column names
    column_names = [description[0] for description in cursor.description]
    print("Columns:", column_names)
    
    # Print rows
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    
    # Also show a count of total rows
    cursor.execute(f"SELECT COUNT(*) FROM {latest_table};")
    row_count = cursor.fetchone()[0]
    print(f"\nTotal rows in '{latest_table}': {row_count}")
    
     # Add column count
    column_count = len(column_names)
    print(f"Total columns in '{latest_table}': {column_count}")
else:
    print("No subscription_data tables found.")

# Close the connection
conn.close()

All tables in the database:
subscription_data_20230116
metrics_subscription_data_20230116
grouped_metrics_subscription_data_2023-01
grouped_metrics_subscription_data_2023_01

First 5 rows from table 'grouped_metrics_subscription_data_2023_01':
Columns: ['provider', 'is_active', 'is_canceled', 'converted', 'personal_person_geo_country', 'price_tier', 'conversion_speed_category', 'conversion_cohort', 'signup_cohort', 'customer_count', 'total_charges_sum', 'avg_total_charges', 'avg_current_mrr', 'avg_payment_periods', 'avg_of_avg_monthly_payment', 'sum_avg_monthly_payment', 'avg_expansion_ratio', 'avg_conversion_speed']
('apple', 1, 0, 1, 'United States of America', '$20+', '1-7 days', '2022-11', '2022-11', 556, 33362.43, 60.00437050359712, 5.0, 2.0017985611510793, 29.981807553956838, 16669.885000000002, 0.16679149706801064, 6.996402877697841)
('apple', 1, 0, 1, 'United States of America', '$17.50-$20', '1-7 days', '2022-10', '2022-10', 553, 33174.47, 59.99, 5.0, 3.0, 19.996666666666666, 